# Chat models
source: https://python.langchain.com/docs/how_to/#chat-models

Langchain **chat models** are language models that use a sequence of messages as inputs and return messages as outputs (as opposed to using plain text). These are generally newer models.

## Setup

Install LLM models as langchain dependencies: 

```bash
uv pip install "langchain[anthropic]"
```

Ex: anthropic, openai, google-genai (gemini), google-vertexai, aws,  groq, cohere, langchain-nvidia-ai-endpoints (nvidia), fireworks, mistralai, together, langchain-ibm (WatsonX), databricks-langchain, langchain-xai, langchain-perplexity, langchain-deepseek, langchain-ollama, ...


See link above for model features.

## Fundamentals of chat models

In [ ]:
# OpenAI chat model example

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
message = "hi! Tell me a joke."

chat_model = ChatOpenAI(openai_api_key=api_key)
try:
    result = chat_model.predict(message)
except Exception as e:
    result = str(e)
    
print(result)

Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [ ]:
#  Generic model example

import getpass
import os
from langchain.chat_models import init_chat_model


if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

llm.invoke("Hello, world!")

In [ ]:
# ---- Anthropic Chat Model Example ----
# Anthropic models: https://docs.anthropic.com/en/docs/models-overview
from langchain.chat_models import ChatAnthropic

model = ChatAnthropic(model="claude-3-opus-20240229")
result = model.invoke(message)
print(f"Answer from Anthropic: {result.content}")


# ---- Google Chat Model Example ----
# https://ai.google.dev/gemini-api/docs/models/gemini
from langchain_googlegenerativeai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = model.invoke(message)
print(f"Answer from Google: {result.content}")

# ---- Ollama chat model example ----
from langchain_ollama import ChatOllama

# Instantiation
model = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

# Invocation
result = model.invoke(message)
print(result.content)

## Chat model used for conversation (sequential inputs)

This introduces the message class and 3 types of messages:  
- SystemMessage: directed to the LLM
- HumanMessage: discussion from human
- AIMessage: discussion from AI LLM

In [ ]:

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Create a ChatOpenAI model

model = ChatOpenAI(model="gpt-4o")

# SystemMessage: Message for priming AI behavior, usually passed in as the first of a sequenc of input messages.
# HumanMessagse: Message from a human to the AI model.
messages = [
    SystemMessage(content="You are a kind teacher that helps students with math problems and explain solutions in a pedagogic way."),
    HumanMessage(content="What is 27 divided by 3?"),
]

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")


# Additional interaction via contextual messages
messages.append(AIMessage(content="That is uncorrect, would you like to know why?"))
messages.append(HumanMessage(content="yes, please."))

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")


## Chat history

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Load environment variables from .env (in particular loads OPENAI_API_KEY)
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini")

# Use a list to store messages
chat_history = []  

# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)

# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    print(f"AI: {response}")


print("---- Message History ----")
print(chat_history)


## Storing chat history in a database
source: https://python.langchain.com/docs/integrations/memory/

In [ ]:
# source: https://python.langchain.com/docs/integrations/memory/google_firestore_datastore/
from dotenv import load_dotenv
import redis
from langchain_google_memorystore_redis import MemorystoreChatMessageHistory
from langchain_openai import ChatOpenAI

load_dotenv()

# Connect to a Memorystore for Redis instance
# docker run -d --name redis -p 6379:6379 redis:alpine 
redis_client = redis.Redis(host='localhost', port=6379)
# redis_client = redis.from_url("redis://127.0.0.1:6379")
redis_key = "Session1"

# Initialize message history
message_history = MemorystoreChatMessageHistory(redis_client, session_id=redis_key)
print("Current Chat History:", message_history.messages)

# Initialize Chat Model
model = ChatOpenAI()
print("Start chatting with the AI. Type 'exit' to quit.")

while True:
    human_input = input("User: ")
    if human_input.lower() == "exit":
        break

    message_history.add_user_message(human_input)

    ai_response = model.invoke(message_history.messages)
    message_history.add_ai_message(ai_response.content)

    print(f"AI: {ai_response.content}")

message_history = MemorystoreChatMessageHistory(redis_client, session_id="session1")
print("Current Chat History:", message_history.messages)

# output:
# Current Chat History: [HumanMessage(content='this is a test', additional_kwargs={}, response_metadata={}), 
#                       HumanMessage(content='Tell me a joke', additional_kwargs={}, response_metadata={}),
#                       AIMessage(content="Why don't scientists trust atoms? Because they make up everything!", additional_kwargs={}, response_metadata={})
# ]